## word2vec

## 추론 기반 기법과 신경망
신경망에서의 단어 처리

In [6]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]]) ## 미니배치 처리를 고려하여 2차원으로 구성
W = np.random.randn(7, 3)
h = np.matmul(c, W)
print(h) ## 당연히 교재와 다른 값

[[-0.81562768 -1.87179517 -0.92998118]]


In [7]:
from common.layers import MatMul

c = np.array([[1,0,0,0,0,0,0]]) ## 미니배치 처리를 고려하여 2차원으로 구성
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-1.91846794  0.25902595  1.72688552]]


## 단순한 word2vec

CBOW 모델의 추론 처리

In [8]:
class Matmul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x, self.W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

In [9]:
c0 = np.array([[1,0,0,0,0,0,0]])
c1 = np.array([[0,0,1,0,0,0,0]])

W_in = np.random.randn(7,3)
W_out = np.random.randn(3,7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[ 1.37306235 -1.38456602  1.30288812 -1.28788019  0.69408365  0.44915833
  -0.17405791]]
